### Cleaning insider trading data

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
csv_file = 'C:\\Users\\hai74\\Desktop\\lit_nonderiv.csv'
db_file = '../../datasets/insider_trading.db'
table_name = 'nonderiv'

# Set the chunk size (number of rows per chunk, you can adjust this as needed)
chunk_size = 100000  # Adjust depending on memory constraints

# Create a connection to the SQLite database
conn = sqlite3.connect(db_file)

# Use the first chunk to create the table, then append subsequent chunks
for i, chunk in enumerate(pd.read_csv(csv_file, chunksize=chunk_size)):
    if i == 0:
        # Create table in the first chunk
        chunk.to_sql(table_name, conn, if_exists='replace', index=False)
    else:
        # Append subsequent chunks to the table
        chunk.to_sql(table_name, conn, if_exists='append', index=False)
    
    print(f"Chunk {i + 1} inserted.")


In [ ]:
# add isBuy col
alter_table_query = f"""
ALTER TABLE {table_name}
ADD COLUMN isBuy INTEGER;
"""
try:
    conn.execute(alter_table_query)
    print(f"'isBuy' column successfully added to {table_name}.")
except sqlite3.OperationalError as e:
    if 'duplicate column name' in str(e).lower():
        print("'isBuy' column already exists.")
    else:
        raise  # Raise other errors if they occur

# Update the 'isBuy' column based on 'transactionCode'
update_isBuy_query = f"""
UPDATE {table_name}
SET isBuy = CASE
    WHEN transactionCode IN ('P', 'A', 'M', 'I', 'G', 'L') THEN 1  -- Buy codes
    WHEN transactionCode IN ('S', 'D', 'F', 'W', 'U') THEN 0  -- Sell codes
    ELSE NULL  -- Set to NULL for undefined codes
END;
"""

conn.execute(update_isBuy_query)
conn.commit()  # Commit the changes

'isBuy' column successfully added to nonderiv.


In [ ]:
# query to select filingDate and isBuy where isBuy is not NULL
select_date_and_buy_query = f"""
SELECT transactionDate, isBuy FROM {table_name}
WHERE isBuy IS NOT NULL;
"""

# Execute the query and load the result into a Pandas DataFrame
df = pd.read_sql_query(select_date_and_buy_query, conn)

In [ ]:
df.set_index('transactionDate', inplace=True)

In [ ]:
df

,isBuy
transactionDate,
2003-05-05,1
2003-05-05,1
2002-10-28,1
2002-12-30,1
2002-12-18,1
...,...
2024-10-16,0
2024-10-16,1
2024-10-16,0


In [ ]:
df.to_csv('../datasets/insider_nonderiv.csv')

In [ ]:
conn.close()